# Colab quickstart (clone repo + cài dependencies)

> Nếu bạn chạy trên **Google Colab**, hãy chạy **2 cell đầu tiên** để tự động:
- clone repo về `/content/23CLCT2_TraditionalMedicineChatbot`
- `pip install -r requirements.txt`
- set `PROJECT_ROOT` và `sys.path` để import `modules.*` chạy được

> Nếu bạn chạy **local VS Code/Windows**, bạn có thể bỏ qua phần clone/pip bên dưới.

In [ ]:
# Auto setup for Colab: clone repo + install requirements + set PROJECT_ROOT/sys.path
import os
import sys
from pathlib import Path

IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ
print('IN_COLAB:', IN_COLAB)

# Repo location on Colab
DEFAULT_PROJECT_ROOT = Path('/content/23CLCT2_TraditionalMedicineChatbot')
PROJECT_ROOT = Path(os.environ.get('PROJECT_ROOT', str(DEFAULT_PROJECT_ROOT))).resolve()

# Default repo URL (override by setting env REPO_URL if needed)
DEFAULT_REPO_URL = 'https://github.com/HuyTran28/23CLCT2_TraditionalMedicineChatbot.git'
REPO_URL = os.environ.get('REPO_URL', DEFAULT_REPO_URL)

if IN_COLAB:
    import subprocess
    if not PROJECT_ROOT.exists():
        print('PROJECT_ROOT does not exist; cloning repo...')
        subprocess.check_call(['git', 'clone', REPO_URL, str(PROJECT_ROOT)])
        print('Cloned to:', PROJECT_ROOT)
    else:
        print('Repo already exists at:', PROJECT_ROOT)
        # Make sure repo is up-to-date (important if you edited code then re-ran on Colab)
        try:
            subprocess.check_call(['git', '-C', str(PROJECT_ROOT), 'pull', '--ff-only'])
        except Exception as e:
            print('git pull failed (non-fatal):', e)

    # Print current commit (helps debugging)
    try:
        head = subprocess.check_output(['git', '-C', str(PROJECT_ROOT), 'rev-parse', '--short', 'HEAD']).decode().strip()
        print('Repo HEAD:', head)
    except Exception:
        pass

    # Install dependencies
    req_file = PROJECT_ROOT / 'requirements.txt'
    if req_file.exists():
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'pip'])
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', '-r', str(req_file)])
        # Optional: for faster/larger models on Colab (4-bit)
        # subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'accelerate', 'bitsandbytes'])
    else:
        raise RuntimeError(f'Không thấy requirements.txt tại: {req_file}')

# Make imports work: add chatbot/ to sys.path
chatbot_dir = (PROJECT_ROOT / 'chatbot').resolve()
if chatbot_dir.exists():
    if str(chatbot_dir) not in sys.path:
        sys.path.insert(0, str(chatbot_dir))
else:
    # Local VS Code: fallback to current-working-directory search
    p = Path.cwd().resolve()
    for _ in range(6):
        if (p / 'chatbot').exists():
            chatbot_dir = (p / 'chatbot').resolve()
            if str(chatbot_dir) not in sys.path:
                sys.path.insert(0, str(chatbot_dir))
            break
        p = p.parent
    else:
        raise RuntimeError('Không tìm thấy thư mục chatbot/. Hãy mở notebook từ trong repo hoặc set env PROJECT_ROOT đúng.')

print('PROJECT_ROOT:', PROJECT_ROOT)
print('chatbot_dir:', chatbot_dir)

# Default env for this notebook
os.environ.setdefault('LLM_BACKEND', 'hf')
os.environ.setdefault('EXTRACTOR_BACKEND', 'hf')
if IN_COLAB:
    os.environ.setdefault('HF_MODEL', 'Qwen/Qwen2.5-7B-Instruct')
else:
    os.environ.setdefault('FORCE_CPU', '1')
    os.environ.setdefault('HF_MODEL', 'Qwen/Qwen2.5-3B-Instruct')

print('HF_MODEL:', os.environ.get('HF_MODEL'))
print('FORCE_CPU:', os.environ.get('FORCE_CPU'))

# Chạy self-hosted trên RTX 3050 (Windows)

Notebook này chạy **self-hosted** bằng HuggingFace (không gọi API LLM bên ngoài).

## Về 4-bit (`load_in_4bit=True`)

- Trên **Windows native**, `bitsandbytes` 4-bit thường không hoạt động ổn định.

- Nếu bạn muốn chạy 7B/14B với 4-bit, khuyến nghị dùng **WSL2 (Ubuntu)** hoặc **Google Colab**.

In [ ]:
# Colab setup (chạy trên Colab để không dùng GPU máy local)

# Nếu bạn đang chạy local VS Code thì cell này có thể bỏ qua.

import os



IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ

print('IN_COLAB:', IN_COLAB)



# Luôn self-host HF

os.environ.setdefault('LLM_BACKEND', 'hf')

os.environ.setdefault('EXTRACTOR_BACKEND', 'hf')



if IN_COLAB:

    # Colab: dùng GPU + có thể 4-bit

    os.environ.setdefault('HF_MODEL', 'Qwen/Qwen2.5-7B-Instruct')

    # (khuyến nghị) cache vào Drive nếu bạn mount Drive:

    # os.environ.setdefault('HF_HOME', '/content/drive/MyDrive/hf_cache')

else:

    # Local: ép CPU để không tiêu GPU máy bạn

    os.environ.setdefault('FORCE_CPU', '1')

    os.environ.setdefault('HF_MODEL', 'Qwen/Qwen2.5-3B-Instruct')



print('HF_MODEL:', os.environ.get('HF_MODEL'))

print('FORCE_CPU:', os.environ.get('FORCE_CPU'))


In [ ]:
# Kiểm tra CUDA + VRAM để chọn cấu hình hợp lý

import os

import torch



print('torch:', torch.__version__)

print('cuda available:', torch.cuda.is_available())

if torch.cuda.is_available():

    print('gpu:', torch.cuda.get_device_name(0))

    props = torch.cuda.get_device_properties(0)

    vram_gb = props.total_memory / (1024**3)

    print(f'vram: {vram_gb:.2f} GB')



# Gợi ý model mặc định theo VRAM (bạn có thể sửa lại)

if os.environ.get('HF_MODEL') is None:

    if torch.cuda.is_available():

        # an toàn cho RTX 3050 phổ biến (4-6GB)

        os.environ['HF_MODEL'] = 'Qwen/Qwen2.5-3B-Instruct'

    else:

        os.environ['HF_MODEL'] = 'Qwen/Qwen2.5-1.5B-Instruct'



os.environ.setdefault('LLM_BACKEND', 'hf')

os.environ.setdefault('EXTRACTOR_BACKEND', 'hf')

print('HF_MODEL:', os.environ['HF_MODEL'])


In [ ]:
# (Colab) Tuỳ chọn: cài thêm nếu muốn chạy 4-bit
# - Nếu bạn đã chạy cell "Auto setup for Colab" ở đầu notebook thì thường đã cài requirements rồi.
# - Cell này chỉ cần khi bạn muốn 4-bit (cần accelerate + bitsandbytes).

import os
import sys
from pathlib import Path

IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ
if not IN_COLAB:
    print('Not in Colab; skip')
else:
    project_root = Path(os.environ.get('PROJECT_ROOT', '/content/23CLCT2_TraditionalMedicineChatbot')).resolve()
    req = project_root / 'requirements.txt'
    if req.exists():
        import subprocess
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'pip'])
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', '-r', str(req)])
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'accelerate', 'bitsandbytes'])
        print('Installed accelerate + bitsandbytes')
    else:
        raise RuntimeError(f'Không thấy requirements.txt tại: {req}')

In [ ]:
import os



# Nếu chạy notebook độc lập trên Colab, hãy mount repo hoặc copy folder chatbot/ lên runtime

# Ví dụ: from google.colab import drive; drive.mount('/content/drive')



# Cấu hình model self-hosting

# Gợi ý: trên Windows + RTX 3050, ưu tiên model nhỏ nếu VRAM không nhiều

os.environ.setdefault('LLM_BACKEND', 'hf')

os.environ.setdefault('EXTRACTOR_BACKEND', 'hf')

os.environ.setdefault('HF_MODEL', os.environ.get('HF_MODEL', 'Qwen/Qwen2.5-3B-Instruct'))


In [ ]:
# Thiết lập sys.path để import được modules.*

import sys

from pathlib import Path



repo_root = Path.cwd()

p = repo_root

for _ in range(6):

    if (p / 'chatbot').exists():

        repo_root = p

        break

    p = p.parent



chatbot_dir = (repo_root / 'chatbot').resolve()

if str(chatbot_dir) not in sys.path:

    sys.path.insert(0, str(chatbot_dir))



print('chatbot_dir:', chatbot_dir)


## Chạy extraction trên Colab và mang JSONL về máy



Mục tiêu: chạy LLM **trên Colab GPU**, xuất ra file JSONL (và ảnh nếu bật), sau đó bạn tải JSONL về máy để chạy ingest/webapp bình thường.


In [ ]:
# (Colab) Mount Drive để lưu output (khuyến nghị)

# Nếu không muốn dùng Drive, bạn có thể bỏ qua cell này.

import os



IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ

if IN_COLAB:

    try:

        from google.colab import drive  # type: ignore

        drive.mount('/content/drive')

        print('Drive mounted at /content/drive')

    except Exception as e:

        print('Could not import google.colab.drive:', e)

else:

    print('Not in Colab; skip drive mount')


In [ ]:
# Chọn đường dẫn project trên Colab

# - Nếu bạn clone repo vào /content/23CLCT2_TraditionalMedicineChatbot thì để nguyên.

# - Nếu bạn để trong Drive, sửa PROJECT_ROOT cho đúng.

import os

import sys

from pathlib import Path



IN_COLAB = 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ



PROJECT_ROOT = Path(os.environ.get('PROJECT_ROOT', '/content/23CLCT2_TraditionalMedicineChatbot')).resolve()



if not PROJECT_ROOT.exists():

    print('PROJECT_ROOT does not exist:', PROJECT_ROOT)

    if IN_COLAB:

        # Tự động clone repo nếu đang chạy trên Colab

        default_repo = 'https://github.com/HuyTran28/23CLCT2_TraditionalMedicineChatbot.git'

        repo_url = os.environ.get('REPO_URL', default_repo)

        print('Cloning from:', repo_url)

        import subprocess

        subprocess.check_call(['git', 'clone', repo_url, str(PROJECT_ROOT)])

        print('Cloned to', PROJECT_ROOT)

    else:

        raise RuntimeError('Không tìm thấy PROJECT_ROOT. Hãy set env PROJECT_ROOT trỏ đúng thư mục repo.')



print('PROJECT_ROOT:', PROJECT_ROOT)



# Add chatbot/ to sys.path so `from modules...` works

chatbot_dir = (PROJECT_ROOT / 'chatbot').resolve()

if not chatbot_dir.exists():

    raise RuntimeError(f'Không thấy thư mục chatbot/: {chatbot_dir}')



if str(chatbot_dir) not in sys.path:

    sys.path.insert(0, str(chatbot_dir))



print('chatbot_dir:', chatbot_dir)


In [ ]:
# (Tùy chọn) Nếu chưa có source trên Colab: clone repo

# Chạy cell này nếu PROJECT_ROOT chưa tồn tại.

import os

from pathlib import Path



PROJECT_ROOT = Path(os.environ.get('PROJECT_ROOT', '/content/23CLCT2_TraditionalMedicineChatbot')).resolve()



if not PROJECT_ROOT.exists():

    # Repo của bạn

    default_repo = 'https://github.com/HuyTran28/23CLCT2_TraditionalMedicineChatbot.git'

    REPO_URL = os.environ.get('REPO_URL', default_repo)

    print('Cloning from:', REPO_URL)

    import subprocess

    subprocess.check_call(['git', 'clone', REPO_URL, str(PROJECT_ROOT)])

    print('Cloned to', PROJECT_ROOT)

else:

    print('Repo already exists at', PROJECT_ROOT)


In [ ]:
# Chạy extraction batch và ghi JSONL
from modules.extractor import MedicalDataExtractor
from modules.ingest_pipeline import iter_markdown_files, iter_chunks_from_file, extract_chunks_to_jsonl
from schemas import medical_schemas
from pathlib import Path
import os

assert 'chatbot_dir' in globals(), 'Hãy chạy cell set PROJECT_ROOT trước'

# Chọn schema + input
SCHEMA_NAME = os.environ.get('SCHEMA_NAME', 'EmergencyProtocol')
INPUT_PATH = os.environ.get('INPUT_PATH', str((Path(chatbot_dir) / 'data' / 'raw').resolve()))

schema = getattr(medical_schemas, SCHEMA_NAME)
print('Schema:', SCHEMA_NAME)
print('Input:', INPUT_PATH)

# Output JSONL: lưu vào Drive nếu có, không thì lưu /content
if IN_COLAB and Path('/content/drive').exists():
    out_dir = Path(os.environ.get('OUT_DIR', '/content/drive/MyDrive/tm_outputs')).resolve()
else:
    out_dir = Path(os.environ.get('OUT_DIR', '/content/tm_outputs')).resolve()
out_dir.mkdir(parents=True, exist_ok=True)

OUT_JSONL = str(out_dir / f'extracted_{SCHEMA_NAME}.jsonl')
print('OUT_JSONL:', OUT_JSONL)

# Khởi tạo extractor (tương thích cả code cũ/mới)
os.environ.setdefault('EXTRACTOR_BACKEND', 'hf')
os.environ.setdefault('LLM_BACKEND', 'hf')

try:
    # Newer version supports backend=...
    extractor = MedicalDataExtractor(backend='hf', model=os.environ['HF_MODEL'], load_in_4bit=None)
except TypeError:
    # Older version: no backend kwarg; rely on env vars
    extractor = MedicalDataExtractor(model=os.environ['HF_MODEL'])

# Chạy trên tất cả file .md trong INPUT_PATH
total_ok = 0
for fp in iter_markdown_files(INPUT_PATH):
    chunks = iter_chunks_from_file(fp, schema, chunk_by='book')
    n_ok = extract_chunks_to_jsonl(
        extractor=extractor,
        chunks=chunks,
        schema=schema,
        out_jsonl_path=OUT_JSONL,
        requests_per_minute=float(os.environ.get('RPM', '2')),
        enrich_images=(os.environ.get('ENRICH_IMAGES', '0') in {'1','true','yes'}),
        resume=True,
    )
    print('Extracted', n_ok, 'records from', fp)
    total_ok += n_ok

print('DONE. Total extracted:', total_ok)

In [ ]:
# (Colab) Zip + download JSONL để mang về máy

from pathlib import Path



out_jsonl = Path(OUT_JSONL)

assert out_jsonl.exists(), f'Không thấy file: {out_jsonl}'



zip_path = out_jsonl.with_suffix('.zip')

import zipfile

with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as z:

    z.write(out_jsonl, arcname=out_jsonl.name)



print('Wrote:', zip_path)



if IN_COLAB:

    try:

        from google.colab import files  # type: ignore

        files.download(str(zip_path))

    except Exception as e:

        print('Could not import google.colab.files:', e)

        print('Zip is at:', zip_path)

else:

    print('Not in Colab; file is at:', zip_path)


In [ ]:
# (Local) Test nhanh 1 đoạn text (tùy chọn)

# Cell này chủ yếu để test nhanh khi chạy local; trên Colab bạn có thể bỏ qua.

import os

from modules.extractor import MedicalDataExtractor

from schemas.medical_schemas import MedicinalPlant



extractor = MedicalDataExtractor(backend='hf', model=os.environ['HF_MODEL'], load_in_4bit=None)



sample_text = '''

1. CÂY BÁCH XÙ



(Tên khác : Cốt tía)

Cây Bách xù thân gỗ nhỏ, cao từ 3 – 4 mét, thân tròn hoặc hơi vuông, cành nhỏ. Ở cành non, lá có hình kim...

'''



plant = extractor.extract_single(

    text=sample_text,

    schema=MedicinalPlant,

    context_hint='From chapter on decorative medicinal plants',

)

plant.model_dump()
